In [1]:
import os
os.sys.path
import cv2                 
import numpy as np         
from random import shuffle 
from tqdm import tqdm      

In [2]:
TRAIN_DIR = '/home/hr/Documents/Train'
IMG_SIZE = 100
LR = 1e-3
MODEL_NAME = 'cnnback100-{}-{}.model'.format(LR, 'conv-net') 
def label_img(folder):
    if folder == 'ALB': return [1,0,0,0,0,0]
    elif folder == 'BET': return [0,1,0,0,0,0]
    elif folder == 'DOL': return [0,0,1,0,0,0]
    elif folder == 'LAG': return [0,0,0,1,0,0]
    elif folder == 'SHARK': return [0,0,0,0,1,0]
    elif folder == 'YFT': return [0,0,0,0,0,1]

In [3]:
from skimage import filters
from scipy import ndimage as ndi
from skimage import morphology

In [4]:
training_data3=[]
for folder in tqdm(os.listdir(TRAIN_DIR)):
    for img in tqdm(os.listdir(TRAIN_DIR+"/"+folder)):
        label=label_img(folder)
        path = os.path.join(TRAIN_DIR+"/"+folder,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        #background
        sobel = filters.sobel(img)
        blur = filters.gaussian(sobel, sigma=1.0)
        light_spots = np.array((img > 240).nonzero()).T
        dark_spots = np.array((img < 60).nonzero()).T
        bool_mask = np.zeros(img.shape, dtype=np.bool)
        bool_mask[tuple(light_spots.T)] = True
        bool_mask[tuple(dark_spots.T)] = True
        seed_mask, num_seeds = ndi.label(bool_mask)
        ws = morphology.watershed(blur, seed_mask)
        background = max(set(ws.ravel()), key=lambda g: np.sum(ws == g))
        background_mask = (ws == background)
        cleanedimg = img * ~background_mask
        
        training_data3.append([np.array(cleanedimg),np.array(label)])
        
        #rotate
        r = 180
        m= cv2.getRotationMatrix2D((IMG_SIZE/2,IMG_SIZE/2),r,1)
        dst = cv2.warpAffine(cleanedimg,m,(IMG_SIZE,IMG_SIZE))
        training_data3.append([np.array(dst),np.array(label)])
        r1 = 90
        m1= cv2.getRotationMatrix2D((IMG_SIZE/2,IMG_SIZE/2),r1,1)
        dst1 = cv2.warpAffine(cleanedimg,m1,(IMG_SIZE,IMG_SIZE))
        training_data3.append([np.array(dst1),np.array(label)])
        r2 = 90
        m2= cv2.getRotationMatrix2D((IMG_SIZE/2,IMG_SIZE/2),r2,1)
        dst2 = cv2.warpAffine(dst,m2,(IMG_SIZE,IMG_SIZE))
        training_data3.append([np.array(dst2),np.array(label)])
        
        

shuffle(training_data3)
np.save('train_data100back.npy', training_data3)

 24%|██▍       | 411/1719 [00:21<01:09, 18.89it/s]


 47%|████▋     | 808/1719 [00:43<00:48, 18.77it/s]


 71%|███████▏  | 1227/1719 [01:04<00:25, 18.96it/s]


 93%|█████████▎| 1597/1719 [01:25<00:06, 18.70it/s]


 12%|█▏        | 87/734 [00:05<00:39, 16.40it/s]


 58%|█████▊    | 427/734 [00:28<00:20, 14.76it/s]


 99%|█████████▉| 730/734 [00:53<00:00, 13.68it/s]


 47%|████▋     | 83/176 [00:04<00:05, 17.57it/s]


100%|██████████| 6/6 [03:02<00:00, 30.45s/it]


In [5]:
import tensorflow as tf
tf.reset_default_graph()

import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = conv_2d(convnet, 32, 3, activation='relu')
convnet = max_pool_2d(convnet, 3, strides =2)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = conv_2d(convnet, 64, 3, activation='relu')
convnet = max_pool_2d(convnet, 3, strides =2)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = conv_2d(convnet, 128, 3, activation='relu')
convnet = max_pool_2d(convnet, 3, strides =2)

convnet = conv_2d(convnet, 256, 5, activation='relu')
convnet = conv_2d(convnet, 256, 3, activation='relu')
convnet = max_pool_2d(convnet, 3, strides =2)

convnet = conv_2d(convnet, 384, 5, activation='relu')
convnet = conv_2d(convnet, 384, 3, activation='relu')
convnet = max_pool_2d(convnet, 3, strides =2)

convnet = conv_2d(convnet, 512, 5, activation='relu')
convnet = conv_2d(convnet, 512, 3, activation='relu')
convnet = max_pool_2d(convnet, 3, strides =2)
convnet = dropout(convnet, 0.4)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = fully_connected(convnet,6 , activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [ ]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')
train1 = np.load("/home/hr/train_data100back.npy")
train = train1[:]
test = train1[-4000:]

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

model.fit(X, Y, n_epoch=50, validation_set=(test_x, test_y), show_metric=True, snapshot_step=200, run_id=MODEL_NAME)

Training Step: 1250  | total loss: 9.38995 | time: 6443.866s
| Adam | epoch: 007 | loss: 9.38995 - acc: 0.5922 -- iter: 07424/12052
